In [4]:
import urllib.request
import zipfile
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [6]:
TRAINING_DIR = 'data/horse-or-human'
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   shear_range=.2,
                                   zoom_range=.2,
                                   width_shift_range=.2,
                                   height_shift_range=.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, batch_size=10, class_mode='binary', target_size=(150, 150))

Found 1027 images belonging to 2 classes.


In [7]:
VALIDATION_DIR = 'data/validation-horse-or-human'
validation_datagen = ImageDataGenerator(rescale = 1/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, batch_size=10, class_mode='binary', target_size = (150, 150))


Found 256 images belonging to 2 classes.


In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu', kernel_initializer='lecun_uniform'),
        tf.keras.layers.Dense(1, activation='sigmoid')
        ])

In [10]:
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

In [11]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('acc') > 0.83) and (logs.get('val_acc') > 0.83):
            self.model.stop_training = True

In [12]:
callbacks = myCallback()

In [13]:
model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[callbacks])

Epoch 1/50
103/103 [==============================] - 37s 345ms/step - loss: 0.6349 - acc: 0.6241 - val_loss: 0.5658 - val_acc: 0.8086
Epoch 2/50
103/103 [==============================] - 37s 355ms/step - loss: 0.5417 - acc: 0.7459 - val_loss: 0.8395 - val_acc: 0.7305
Epoch 3/50
103/103 [==============================] - 36s 350ms/step - loss: 0.4311 - acc: 0.8130 - val_loss: 0.6978 - val_acc: 0.8359
Epoch 4/50
103/103 [==============================] - 37s 355ms/step - loss: 0.3868 - acc: 0.8238 - val_loss: 0.7124 - val_acc: 0.8359
Epoch 5/50
103/103 [==============================] - 37s 360ms/step - loss: 0.3474 - acc: 0.8588 - val_loss: 0.5181 - val_acc: 0.8594
